Connect to drive, set directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/MyDrive/Thesis_UU/3group'

Import libraries

In [ ]:
import pandas as pd
from collections import Counter

Load Retraction Watch database

In [ ]:
retracted_db = pd.read_excel('Data (Excel)/Retracted_Database.xlsx')

In [ ]:
retracted_db.rename(columns={"Record ID": "Record_ID"}, inplace = True)

In [ ]:
retracted_db.Reason.head(5)

In [ ]:
len(retracted_db)

In [ ]:
retracted_db.Reason = retracted_db.Reason.str.lower() #lowercase reasons for retraction

In [ ]:
retracted_db.Reason[1] #check data

In [ ]:
retracted_db.Reason[12]

In [ ]:
retracted_db[retracted_db.Reason.str.contains('error|duplication of', na=False)].head(2) #check data

In [ ]:
retracted_db.Reason = retracted_db.Reason.str.replace(';', ',', regex = True) #clean up colun 'Reason'

In [ ]:
retracted_db.Reason = retracted_db.Reason.str.replace('+', '', regex = True) #clean up colun 'Reason'

Splitting up reasons for retraction, as most papers have more than one reason for retraction listed

In [ ]:
retracted_db.Reason = retracted_db.Reason.map(lambda x: str(x)[:-1]) 

In [ ]:
results = set()
retracted_db['Reason'].str.split(',').apply(results.update)
print(results)

In [ ]:
results #all reasons for retraction

Check how often reasons appear in the database

In [ ]:
results = Counter()
retracted_db['Reason'].str.lower().str.split(',').apply(results.update)
print(results)
#counts number of appearance for every reason

In [ ]:
retracted_db['Reason'].apply(lambda x: pd.value_counts(x.split(','))).sum(axis = 0).sort_values(ascending = False).head(50)
#lists 30 most frequent reasons and their appearance

In [ ]:
retracted_db.Reason.tail(10)

## Create the 2 groups: error and misconduct

In [ ]:
error = retracted_db[retracted_db.Reason.str.contains('error', na=False)]
#subset with every paper that contains a reason which includes the term 'error'

In [ ]:
len(error) #size of subset

In [ ]:
error.Reason.head(5)

In [ ]:
error['Reason'].apply(lambda x: pd.value_counts(x.split(','))).sum(axis = 0).sort_values(ascending = False).head(30)
#lists of top 30 most frequent reasons for retraction in error group

In [ ]:
misconduct = retracted_db[retracted_db.Reason.str.contains('fake|false|falsification|hoax|manipulation|misconduct by|paper mill|plagiarism of|randomly generated content|sabotage|salami slicing', na=False)]
#subset with every paper that contains a reason selected for misconduct group

In [ ]:
len(misconduct) #size of subset

In [ ]:
misconduct.Reason.head()

In [ ]:
misconduct['Reason'].apply(lambda x: pd.value_counts(x.split(','))).sum(axis = 0).sort_values(ascending = False).head(30)
#lists of top 30 most frequent reasons for retraction in misconduct group

Remove overlapping papers / papers that contain both error and misconduct reasons

In [ ]:
overlap = pd.merge(misconduct, error, how='inner', left_on='Record_ID', right_on='Record_ID')
#create subset containing papers with overlapping reasons

In [ ]:
len(overlap)

In [ ]:
overlap.head(2)

In [ ]:
overlap.Reason_x.head(3)

In [ ]:
error = error[~error.Record_ID.isin(overlap.Record_ID)] #removal of overlap from error subset

In [ ]:
len(error) #size of error group

In [ ]:
misconduct = misconduct[~misconduct.Record_ID.isin(overlap.Record_ID)] #removal of overlap from misconduct subset

In [ ]:
len(misconduct) #size of misconduct group

In [ ]:
error['misconduct'] = 0 #create column indicating group

In [ ]:
error.head(2)

In [ ]:
misconduct['misconduct'] = 1 #create column indicating group

In [ ]:
misconduct.head(2)

Save files

In [ ]:
error.to_csv('Data (CSV)/error_db.csv')

In [ ]:
misconduct.to_csv('Data (CSV)/misconduct_db.csv')